In [1]:
%reset -f
import mne
from mne.io import concatenate_raws as conc
#from braindecode.mne_ext.signalproc import concatenate_raws_with_events as conc
from braindecode.mne_ext.signalproc import resample_cnt as resample
import os

%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
%run analysis_classes.ipynb
infolder, outfolder = find_folder()

# paramters
num=17
freq_rate = 200
low_pass = 0.1
high_pass = 50
im_start, im_end = -0.1, 5
pr_start, pr_end = -0.1, 1

In [2]:
%run analysis_classes.ipynb
files = ['IR_'+str(num)+'_S01.bdf','IR_'+str(num)+'_S02.bdf']
add_files = [os.path.join(infolder,a) for a in files]


# Load each of the files
parts = [mne.io.read_raw_bdf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in add_files]

# Concatenate them
raw = conc(parts)
events = mne.find_events(raw, initial_event=True,consecutive=True, shortest_event=1)
raw.filter(low_pass,high_pass,method='iir')
raw = custom_tools(raw).running_zscore_threshold(window_size=500)
#raw = custom_tools(raw).ems()
raw.set_eeg_reference(ref_channels='average', projection=False, ch_type='auto', verbose=None)

Progress: [####################] 98.5%


/pkgs/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2315: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/pkgs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater
/pkgs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


Creating RawArray with float64 data, n_channels=65, n_times=6775296
    Range : 0 ... 6775295 =      0.000 ... 13232.998 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


<RawArray  |  None, n_channels x n_times : 65 x 6775296 (13233.0 sec), ~3.28 GB, data loaded>

In [3]:
events = events.astype('int')
baseline = (None, 0)  # means from the first instant to t = 0     
#raw.drop_channels(['Status'])

# segmenting imagery
trigs=list(range(31, 36))
event_ids={str(x):x for x in trigs}

ep_img = mne.Epochs(raw, events, event_ids, im_start, im_end,
                baseline=baseline, preload=True)
ep_img.equalize_event_counts(event_ids=event_ids, method='mintime') 
ep_img.resample(freq_rate)
print('Starting saving')
ep_img.save(os.path.join(outfolder,'nn_img_S'+str(num)+'-epo.fif'), overwrite=True)

# segmenting perception
trigs=list(range(101, 151))+list(range(201, 251))
event_ids={str(x):x for x in trigs}

ep_prc = mne.Epochs(raw, events, event_ids, pr_start, pr_end,
                baseline=baseline, preload=True)
ep_prc.equalize_event_counts(event_ids=event_ids, method='mintime') 
ep_prc.resample(freq_rate)

print('Starting saving')
ep_prc.save(os.path.join(outfolder,'nn_prc_S'+str(num)+'-epo.fif'), overwrite=True)

484 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 484 events and 2612 original time points ...
0 bad epochs dropped
Dropped 4 epochs
Starting saving
Overwriting existing file.
5002 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 5002 events and 564 original time points ...
0 bad epochs dropped
Dropped 2 epochs
Starting saving
Overwriting existing file.


In [4]:
import pandas as pd
%run EEG_auxiliary_module.ipynb
im_names = ['mcy','sgo','sjo','est','tsw']
times=[(1,2)]
times=[(0,5),(0,4),(0,3),(0,2),(0,1),(1,2),(1,3),(1,4),(1,5),(2,3),(2,4),(2,5),(3,4),(3,5),(4,5)]
results=[]# Loading first S08 session
durs=[];
for i in times:
    X, Y = convert_epochs_to_2D_array(ep_img, times=i)
    confusion, duration = run_eeg_svm(X,Y,12)
    results.append(confusion)
    durs.append(duration)

os.system('say "your program has finished"')

Progress: [####################] 100.0%
The overall accuracy is 51.9 and the duration is 2.7 minutes


32512

In [5]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
for i in results:
    print(f'Accuracy of the entire set is {np.mean(squareform(i))*100:.2f}%')
confusion_df = pd.DataFrame(results[3], columns = im_names, index = im_names)
#fname = op.join(outfolder,'conf_IR_17_imag.csv')
#confusion_df.to_csv(fname,index = True, header = True)
confusion_df.head()

Accuracy of the entire set is 55.31%
Accuracy of the entire set is 55.73%
Accuracy of the entire set is 55.31%
Accuracy of the entire set is 58.02%
Accuracy of the entire set is 53.49%
Accuracy of the entire set is 57.24%
Accuracy of the entire set is 54.84%
Accuracy of the entire set is 54.43%
Accuracy of the entire set is 54.48%
Accuracy of the entire set is 53.23%
Accuracy of the entire set is 51.20%
Accuracy of the entire set is 53.12%
Accuracy of the entire set is 49.90%
Accuracy of the entire set is 51.46%
Accuracy of the entire set is 51.88%


,mcy,sgo,sjo,est,tsw
mcy,0.000000,0.494792,0.635417,0.541667,0.536458
sgo,0.494792,0.000000,0.489583,0.552083,0.572917
sjo,0.635417,0.489583,0.000000,0.630208,0.630208
est,0.541667,0.552083,0.630208,0.000000,0.718750
tsw,0.536458,0.572917,0.630208,0.718750,0.000000


In [30]:
%run EEG_auxiliary_module.ipynb
times=[(0.05,0.65)]
results=[]# Loading first S08 session
durs=[];
for i in times:
    X, Y = convert_epochs_to_2D_array(ep_prc, times=i)
    #Y = np.array(convert_lbl(Y))
    confusion, duration = run_eeg_svm(X, Y, 12)
    results.append(confusion)
    durs.append(duration)
    print(i)
    #np.save('/Users/dannem/Documents/Ilya_study/Analysis/conf_IR_02_S01_c.csv',confusion)
os.system('say "your program has finished"')
#results

Progress: [####################] 100.0%
The overall accuracy is 63.9 and the duration is 60.2 minutes
(0.05, 0.65)


1

In [31]:
confusion=results[0]
from scipy.spatial.distance import squareform
print(f'Accuracy of the upright stimuli is {np.mean(squareform(confusion[0:50,0:50]))*100:.4f}%')
print(f'Accuracy of the upright unfamiliary stimuli is {np.mean(squareform(confusion[0:25,0:25]))*100:.4f}%')
print(f'Accuracy of the upright famous stimuli is {np.mean(squareform(confusion[25:50,25:50]))*100:.4f}%')
print(f'Accuracy of the inverted stimuli is {np.mean(squareform(confusion[50:,50:]))*100:.4f}%')
print(f'Accuracy of the inverted unfamiliar stimuli is {np.mean(squareform(confusion[50:75,50:75]))*100:.4f}%')
print(f'Accuracy of the inverted famous stimuli is {np.mean(squareform(confusion[75:,75:]))*100:.4f}%')
print(f'Accuracy of the entire set is {np.mean(squareform(confusion))*100:.4f}%')



Accuracy of the upright stimuli is 55.9711%
Accuracy of the upright unfamiliary stimuli is 57.7948%
Accuracy of the upright famous stimuli is 53.8731%
Accuracy of the inverted stimuli is 57.8500%
Accuracy of the inverted unfamiliar stimuli is 60.1944%
Accuracy of the inverted famous stimuli is 55.4904%
Accuracy of the entire set is 63.9209%
Correlation between identity discrimination in upright and inverted famous faces is 0.397
Correlation between identity discrimination in upright and inverted unfamiliar faces is 0.059


NameError: name 'conf' is not defined

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
import matplotlib.pyplot as plt
mne.viz.plot_raw(raw, n_channels=5, scalings=dict(eeg=20e-1))
